In [1]:
!pip install pytube
!pip install yt_dlp
# !pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.5 MB/s eta 0:00:00


In [ ]:
# from google.colab import drive
# import os
# drive.mount('/content/gdrive')
# if os.path.isdir('/content/gdrive/MyDrive/RGB_data_stream'):
#   print("Dataset folder mounted")
# else:
#   print("Error condition, not found folder")
# !ls /content/gdrive/MyDrive/RGB_data_stream
# !ls /content/gdrive/MyDrive/RGB_data_stream/data
# base_folder = '/content/gdrive/MyDrive/RGB_data_stream'

In [7]:
import shutil
import time

# Mount Google Drive
from google.colab import drive
import os
import json
import yt_dlp
import random

drive.mount('/content/gdrive')

# Define base folder path
base_folder = '/content/gdrive/MyDrive/RGB_data_stream'
trimmed_videos_dir = os.path.join(base_folder, 'short')
data_folder = os.path.join(base_folder, 'data/3DYoga90.json')

# Minimum free disk space requirement in bytes (25GB)
MIN_FREE_SPACE = 27 * 1024 * 1024 * 1024

# Check if the base folder exists
if os.path.isdir(base_folder):
    print("Dataset folder mounted successfully.")
else:
    raise FileNotFoundError("Error: RGB_data_stream folder not found.")

# Create a directory for trimmed videos if it doesn't exist
if not os.path.exists(trimmed_videos_dir):
    os.makedirs(trimmed_videos_dir)

# Log files
downloaded_log_file = os.path.join(trimmed_videos_dir, 'downloaded_log.txt')
missing_log_file = os.path.join(trimmed_videos_dir, 'missing_videos_log.txt')

# Initialize log files if they don't exist
if not os.path.exists(downloaded_log_file):
    open(downloaded_log_file, 'w').close()

if not os.path.exists(missing_log_file):
    open(missing_log_file, 'w').close()

# Load JSON dataset
with open(data_folder, 'r') as f:
    data = json.load(f)

# List of poses to download
desired_poses = ['mountain', 'half-way-lift', 'standing-forward-bend', 'downward-dog']

# Helper function to log progress
def log_to_file(file_path, message):
    with open(file_path, 'a') as f:
        f.write(message + '\n')

# Check for processed videos to resume from a save point
with open(downloaded_log_file, 'r') as f:
    downloaded_videos = f.read().splitlines()


# Function to check free disk space
# def check_free_space():
#     total, used, free = shutil.disk_usage(base_folder)
#     print('Total:', total)
#     print('Used:', used)
#     print('Free:', free)
#     return free

# Function to check free disk space
# def check_drive_space():
#   total, used, free = os.popen('df -h /content/drive').read().split()[1:4]
#   return total, used, free


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Dataset folder mounted successfully.


**THIS ONE IS NEW ONE THAT WORKED ON LOCAL MACHINE**

In [6]:
# Main function to process videos with disk space checking and rate limiting
def download_video_segments():
    rate_limit = 1  # seconds between downloads to avoid overloading
    for entry in data:
        pose = entry['pose']
        if pose not in desired_poses:
            continue

        instances = entry['instances']
        for inst in instances:
            sequence_id = str(inst['sequence_id'])

            # Skip if the video has already been processed
            if sequence_id in downloaded_videos:
                print(f'Skipping already processed video with sequence_id: {sequence_id}')
                continue

            # # Check if there's enough free space before downloading
            # free_space = check_drive_space()
            # if free_space < MIN_FREE_SPACE:
            #     print(f"Insufficient disk space! Required: {MIN_FREE_SPACE/(1024**3)}GB, Available: {free_space/(1024**3)}GB")
            #     return

            url = inst['url']
            start_time = inst['frame_start']
            end_time = inst['frame_end']
            video_filename = f"{sequence_id}.mp4"
            output_path = os.path.join(trimmed_videos_dir, video_filename)

            try:
                print(f"Processing video {sequence_id} from {url} [{start_time}s - {end_time}s]")

                ydl_opts = {
                    'format': 'bestvideo[height<=720][ext=mp4]/best[height<=720][ext=mp4]/best',
                    'outtmpl': output_path,
                    'download_ranges': lambda info_dict, yt_instance: [
                        {'start_time': start_time, 'end_time': end_time},
                    ],
                    'force_keyframes_at_cuts': True,
                    'postprocessors': [{
                        'key': 'FFmpegVideoRemuxer',
                        'preferedformat': 'mp4',
                    }],
                    # 'quiet': True,
                    'no_warnings': True,
                }

                with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                    ydl.download([url])

                # Log the successful download
                log_to_file(downloaded_log_file, sequence_id)
                print(f"Downloaded video {sequence_id} successfully.")

                # Rate limiting to avoid overwhelming servers
                # time.sleep(rate_limit)
                time.sleep(random.uniform(2.0, 2.5))

            except Exception as e:
                print(f"Error processing video {sequence_id}: {e}")
                log_to_file(missing_log_file, f"{sequence_id}: {url} - Error: {e}")

# Call the download function
download_video_segments()



Processing video 1000 from https://www.youtube.com/watch?v=5vIJX7SIiJo [159s - 163s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=5vIJX7SIiJo
[youtube] 5vIJX7SIiJo: Downloading webpage
[youtube] 5vIJX7SIiJo: Downloading ios player API JSON
[youtube] 5vIJX7SIiJo: Downloading mweb player API JSON
[youtube] 5vIJX7SIiJo: Downloading m3u8 information
[info] 5vIJX7SIiJo: Downloading 1 format(s): 136
[info] 5vIJX7SIiJo: Downloading 1 time ranges: 159.0-163.0
[download] /content/gdrive/MyDrive/RGB_data_stream/short_vids/1000.mp4 has already been downloaded
[download] 100% of  348.26KiB
[VideoRemuxer] Not remuxing media file "/content/gdrive/MyDrive/RGB_data_stream/short_vids/1000.mp4"; already is in target format mp4
Downloaded video 1000 successfully.
Processing video 1002 from https://www.youtube.com/watch?v=HelFdy8B4X4 [46s - 49s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=HelFdy8B4X4
[youtube] HelFdy8B4X4: Downloading webpage
[youtube] HelFdy8B4X4: Downloading i

ERROR: [youtube] AVqev3Lpas8: Video unavailable


Error processing video 1051: ERROR: [youtube] AVqev3Lpas8: Video unavailable
Processing video 1052 from https://www.youtube.com/watch?v=f2DA84gu10g [404s - 406s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=f2DA84gu10g
[youtube] f2DA84gu10g: Downloading webpage
[youtube] f2DA84gu10g: Downloading ios player API JSON
[youtube] f2DA84gu10g: Downloading mweb player API JSON


ERROR: [youtube] f2DA84gu10g: Video unavailable


Error processing video 1052: ERROR: [youtube] f2DA84gu10g: Video unavailable
Processing video 1053 from https://www.youtube.com/watch?v=iY7LaCQM8_c [162s - 167s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=iY7LaCQM8_c
[youtube] iY7LaCQM8_c: Downloading webpage
[youtube] iY7LaCQM8_c: Downloading ios player API JSON
[youtube] iY7LaCQM8_c: Downloading mweb player API JSON
[youtube] iY7LaCQM8_c: Downloading m3u8 information
[info] iY7LaCQM8_c: Downloading 1 format(s): 135
[info] iY7LaCQM8_c: Downloading 1 time ranges: 162.0-167.0
[download] Destination: /content/gdrive/MyDrive/RGB_data_stream/short_vids/1053.mp4
[download] 100% of  258.64KiB in 00:00:09 at 27.02KiB/s
[VideoRemuxer] Not remuxing media file "/content/gdrive/MyDrive/RGB_data_stream/short_vids/1053.mp4"; already is in target format mp4
Downloaded video 1053 successfully.
Processing video 1054 from https://www.youtube.com/watch?v=zidmxlIy2B4 [121s - 123s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=z

KeyboardInterrupt: 

In [ ]:
import os
import time
import random
import yt_dlp

# Function to download a specific clip from YouTube using ffmpeg
def download_youtube_clip(url, start_time, end_time, output_path):
    ffmpeg_args = {
        "ffmpeg_i": ["-ss", str(start_time), "-to", str(end_time)]
    }

    ydl_opts = {
        'format': 'bestvideo[height<=720]/best',
        'outtmpl': output_path,
        'external_downloader': 'ffmpeg',
        'external_downloader_args': ffmpeg_args,
        'postprocessors': [{
            'key': 'FFmpegVideoRemuxer',
            'preferedformat': 'mp4',
        }],
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
        print(f"Successfully downloaded: {output_path}")
        return True
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        if os.path.exists(output_path):
            os.remove(output_path)
        return False

# Main function to process videos with disk space checking and rate limiting
def download_video_segments():
    rate_limit = 1  # seconds between downloads to avoid overloading
    for entry in data:
        pose = entry['pose']
        if pose not in desired_poses:
            continue

        instances = entry['instances']
        for inst in instances:
            sequence_id = str(inst['sequence_id'])

            # Skip if the video has already been processed
            if sequence_id in downloaded_videos:
                print(f'Skipping already processed video with sequence_id: {sequence_id}')
                continue

            url = inst['url']
            start_time = inst['frame_start']
            end_time = inst['frame_end']
            video_filename = f"{sequence_id}.mp4"
            output_path = os.path.join(trimmed_videos_dir, video_filename)

            try:
                print(f"Processing video {sequence_id} from {url} [{start_time}s - {end_time}s]")

                # Download the specific video segment using the new function
                result = download_youtube_clip(url, start_time, end_time, output_path)
                if result:
                    # Log the successful download
                    log_to_file(downloaded_log_file, sequence_id)
                else:
                    raise Exception("Download failed or was interrupted")

                # Rate limiting to avoid overwhelming servers
                time.sleep(random.uniform(2.0, 2.5))

            except Exception as e:
                print(f"Error processing video {sequence_id}: {e}")
                log_to_file(missing_log_file, f"{sequence_id}: {url} - Error: {e}")

# Call the download function
download_video_segments()


Processing video 1000 from https://www.youtube.com/watch?v=5vIJX7SIiJo [159s - 163s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=5vIJX7SIiJo
[youtube] 5vIJX7SIiJo: Downloading webpage
[youtube] 5vIJX7SIiJo: Downloading ios player API JSON
[youtube] 5vIJX7SIiJo: Downloading mweb player API JSON
[youtube] 5vIJX7SIiJo: Downloading m3u8 information
[info] 5vIJX7SIiJo: Downloading 1 format(s): 136
[download] Destination: /content/gdrive/MyDrive/RGB_data_stream/short/1000.mp4
[download] 100% of  858.81KiB in 00:00:05 at 151.17KiB/s
[VideoRemuxer] Not remuxing media file "/content/gdrive/MyDrive/RGB_data_stream/short/1000.mp4"; already is in target format mp4
Successfully downloaded: /content/gdrive/MyDrive/RGB_data_stream/short/1000.mp4
Processing video 1002 from https://www.youtube.com/watch?v=HelFdy8B4X4 [46s - 49s]
[youtube] Extracting URL: https://www.youtube.com/watch?v=HelFdy8B4X4
[youtube] HelFdy8B4X4: Downloading webpage
[youtube] HelFdy8B4X4: Downloading ios player API

In [5]:
import os
import time
import subprocess

# Function to log messages to a file
def log_to_file(log_file, message):
    with open(log_file, 'a') as f:
        f.write(f"{message}\n")

# Main function to process videos with disk space checking and rate limiting
def download_video_segments():
    rate_limit = 1  # seconds between downloads to avoid overloading
    yt_dlp_log_file = "yt_dlp_output.log"  # File to log yt-dlp output

    for entry in data:
        pose = entry['pose']
        if pose not in desired_poses:
            continue

        instances = entry['instances']
        for inst in instances:
            sequence_id = str(inst['sequence_id'])

            # Skip if the video has already been processed
            if sequence_id in downloaded_videos:
                print(f'Skipping already processed video with sequence_id: {sequence_id}')
                continue

            url = inst['url']
            start_time = inst['frame_start']
            end_time = inst['frame_end']
            video_filename = f"{sequence_id}.mp4"
            output_path = os.path.join(trimmed_videos_dir, video_filename)

            try:
                print(f"Processing video {sequence_id} from {url} [{start_time}s - {end_time}s]")

                ydl_opts = {
                    'format': 'bestvideo[height<=720][ext=mp4]/best[height<=720][ext=mp4]/best',
                    'outtmpl': output_path,
                    'download_ranges': lambda info_dict, yt_instance: [
                        {'start_time': start_time, 'end_time': end_time},
                    ],
                    'force_keyframes_at_cuts': True,
                    'postprocessors': [{
                        'key': 'FFmpegVideoRemuxer',
                        'preferedformat': 'mp4',
                    }],
                    # 'quiet': True,
                    'no_warnings': True,
                }

                # Log output of yt-dlp to the log file
                with open(yt_dlp_log_file, 'a') as log_output:
                    process = subprocess.Popen(
                        ['yt-dlp', url],
                        stdout=log_output,
                        stderr=log_output
                    )
                    process.wait()  # Wait for the process to complete

                if process.returncode == 0:
                    log_to_file(downloaded_log_file, sequence_id)
                    print(f"Downloaded video {sequence_id} successfully.")
                else:
                    raise Exception(f"yt-dlp failed with return code {process.returncode}")

                # Rate limiting to avoid overwhelming servers
                time.sleep(rate_limit)

            except Exception as e:
                print(f"Error processing video {sequence_id}: {e}")
                log_to_file(missing_log_file, f"{sequence_id}: {url} - Error: {e}")

# Call the download function
download_video_segments()


Processing video 1000 from https://www.youtube.com/watch?v=5vIJX7SIiJo [159s - 163s]
Downloaded video 1000 successfully.
Processing video 1002 from https://www.youtube.com/watch?v=HelFdy8B4X4 [46s - 49s]
Downloaded video 1002 successfully.
Processing video 1003 from https://www.youtube.com/watch?v=jMJfl5w30Ds [1859s - 1861s]
Downloaded video 1003 successfully.
Processing video 1004 from https://www.youtube.com/watch?v=AoGt1DKsiDo [747s - 749s]
Downloaded video 1004 successfully.
Processing video 1005 from https://www.youtube.com/watch?v=syXsoVb8NVo [48s - 52s]
Downloaded video 1005 successfully.
Processing video 1006 from https://www.youtube.com/watch?v=YdHbkUB4KhA [41s - 45s]
Downloaded video 1006 successfully.
Processing video 1007 from https://www.youtube.com/watch?v=-r15RxP8ZzE [205s - 209s]
Downloaded video 1007 successfully.
Processing video 1008 from https://www.youtube.com/watch?v=ArDlqoRqfko [82s - 85s]
Downloaded video 1008 successfully.
Processing video 1009 from https://www

KeyboardInterrupt: 

In [ ]:
# def download_video_segments():
#     for entry in data:
#         pose = entry['pose']
#         if pose not in desired_poses:
#             continue

#         instances = entry['instances']
#         for inst in instances:
#             sequence_id = str(inst['sequence_id'])

#             # Skip if the video has already been processed
#             if sequence_id in downloaded_videos:
#                 print(f'Skipping already processed video with sequence_id: {sequence_id}')
#                 continue

#             url = inst['url']
#             start_time = inst['frame_start']
#             end_time = inst['frame_end']
#             video_filename = f"{sequence_id}.mp4"
#             output_path = os.path.join(trimmed_videos_dir, video_filename)

#             try:
#                 print(f"Processing video {sequence_id} from {url} [{start_time}s - {end_time}s]")

#                 # Use yt-dlp to download the specific section using ffmpeg
#                 ydl_opts = {
#                     'format': 'bestvideo[ext=mp4]',  # Download best video, no audio
#                     'outtmpl': output_path,
#                     'quiet': True,  # Avoid excessive log outputs
#                     'no_warnings': True,
#                 }

#                 with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#                     ydl.download([url])

#                 # Use ffmpeg to trim the video to the required section
#                 ffmpeg_command = [
#                     'ffmpeg',
#                     '-i', output_path,
#                     '-ss', str(start_time),  # Start time in seconds
#                     '-to', str(end_time),    # End time in seconds
#                     '-c', 'copy',            # Avoid re-encoding, just copy the stream
#                     output_path
#                 ]

#                 # Run the ffmpeg command
#                 subprocess.run(ffmpeg_command, check=True)

#                 # Log the successful download
#                 log_to_file(downloaded_log_file, sequence_id)
#                 print(f"Downloaded and trimmed video {sequence_id} successfully.")

#                 # Rate limiting to avoid overwhelming servers
#                 time.sleep(random.uniform(1.0, 1.5))

#             except Exception as e:
#                 print(f"Error processing video {sequence_id}: {e}")
#                 log_to_file(missing_log_file, f"{sequence_id}: {url} - Error: {e}")

# # Call the download function
# download_video_segments()